In [1]:
import pandas as pd
# import yfinance as yf


In [88]:
zero = pd.read_csv('./ZeroTrade.csv')
n50 = pd.read_csv('./Nifty50_data.csv',thousands=',')

data = pd.DataFrame(index=zero.index)
data['Stock'] = zero.symbol
data['Price'] = zero.price
data['Qty'] = zero.trade_type.map({'buy': 1, 'sell': -1}) * zero.quantity
data['Amount'] = data['Price'] * data['Qty']
data['cum_qty'] = data.groupby('Stock')['Qty'].cumsum()
data['cum_Amt'] = data.groupby('Stock')['Amount'].cumsum()
data['cum_Avg'] = data.loc[data['cum_qty'] > 0]['cum_Amt'] / data.loc[data['cum_qty'] > 0]['cum_qty']
data['Profit'] = -1 * data.loc[data['cum_qty'] == 0]['cum_Amt']
data['Profit'] = data['Profit'].fillna(0)
data['%Profit'] = -100 * data['Profit'] / data['Amount']
data['%Profit'] = data.groupby('Stock')['%Profit'].cumsum()
data['cum_profit'] = data.groupby('Stock')['Profit'].cumsum()
data['Invested_amt'] = data['cum_Amt'] + data['cum_profit']
data['apparent_avg'] = data['Invested_amt'] / data['cum_qty']
data['apparent_amt'] = data['apparent_avg'] * data['cum_qty']
data = data.fillna(0)


stock_slice = []
for i in data['Stock'].drop_duplicates():
    stock_df = data.loc[data['Stock'] == i]
    ind_x = stock_df[stock_df['cum_qty'].map(lambda x: x == 0)].index
    stock_slice.append(stock_df.tail(1).index[0])


stock_slice = pd.Index(stock_slice)
foli0 = data.iloc[stock_slice]
foli0 = foli0.set_index('Stock')
foli0 = foli0.sort_index()
foli0.loc['Total'] = foli0.sum(numeric_only=True)
foli0.loc['Total', ['Price','Qty','Amount','cum_Avg','Profit','apparent_avg']] = pd.NA
foli0.loc['Total', ['%Profit']] = 100 * foli0.at['Total', 'cum_profit'] / foli0.at['Total', 'Invested_amt']
foli0

,Price,Qty,Amount,cum_qty,cum_Amt,cum_Avg,Profit,%Profit,cum_profit,Invested_amt,apparent_avg,apparent_amt
Stock,,,,,,,,,,,,
BANKBARODA,139.45,5,697.25,5.0,697.25,139.45,0,0.000000,0.00,697.25,139.45,697.25
HDFCBANK,1401.95,1,1401.95,1.0,1401.95,1401.95,0,0.000000,0.00,1401.95,1401.95,1401.95
ICICIBANK,855.15,2,1710.3,7.0,6262.05,894.579,0,0.000000,0.00,6262.05,894.579,6262.05
INFY,1520.1,-2,-3040.2,0.0,-173.00,0,173,5.690415,173.00,0.00,0,0.00
M&M,1334.85,1,1334.85,6.0,7757.60,1292.93,0,0.000000,0.00,7757.60,1292.93,7757.60
NATIONALUM,71.4,10,714,30.0,2189.35,72.9783,0,0.000000,0.00,2189.35,72.9783,2189.35
PNB,36.3,3,108.9,3.0,108.90,36.3,0,0.000000,0.00,108.90,36.3,108.90
RELIANCE,2434,-5,-12170,0.0,15.40,0,-15.4,-0.126541,-15.40,0.00,0,0.00
SBIN,529,2,1058,5.0,2725.80,545.16,0,0.000000,0.00,2725.80,545.16,2725.80


In [89]:
s50 = set(n50.Symbol)
s0 = set(zero.symbol)

In [90]:
s50.intersection(s0)

{'HDFCBANK',
 'ICICIBANK',
 'INFY',
 'M&M',
 'RELIANCE',
 'SBIN',
 'TATAMOTORS',
 'TCS',
 'TECHM',
 'WIPRO'}

In [91]:
s0.difference(s50)

{'BANKBARODA', 'NATIONALUM', 'PNB'}

In [92]:
n50

,Symbol,Open,High,Low,Last Traded Price,Change,%Change,Traded Volume(lacs),Traded Value(crs),52 Week High,52 Week Low,365 Days % Change,30 Days % Change
0,NIFTY 50,18376.40,18399.45,18327.20,18338.70,-11.00,-0.06,971.18,6875.17,18362.30,15183.40,2.60,7.10
1,HINDALCO,435.00,452.85,434.05,449.90,20.05,4.66,114.00,510.87,636.00,308.95,-0.89,12.42
2,TATAMOTORS,424.00,437.45,423.50,435.25,11.75,2.77,74.23,320.82,536.70,366.20,-13.55,9.76
3,APOLLOHOSP,4430.00,4568.00,4425.20,4540.50,119.10,2.69,3.88,175.87,5935.40,3361.55,-0.59,5.91
4,GRASIM,1711.00,1741.90,1709.05,1741.55,33.10,1.94,1.82,31.50,1929.80,1276.60,-5.57,4.18
5,POWERGRID,210.95,213.70,210.70,212.00,2.95,1.41,31.52,66.99,248.35,180.30,16.39,-1.65
6,TATASTEEL,108.50,110.00,108.25,108.85,1.30,1.21,245.64,268.19,138.67,82.70,-91.62,8.25
7,UPL,771.75,785.00,768.65,777.00,9.10,1.19,20.17,156.79,848.00,607.50,1.61,15.35
8,KOTAKBANK,1902.85,1925.00,1901.00,1920.00,20.90,1.10,5.72,109.72,2108.00,1631.00,-6.53,6.07
9,ULTRACEMCO,6845.00,6895.50,6819.40,6886.10,72.90,1.07,1.24,84.83,8172.55,5157.05,-14.47,11.47


In [93]:
slice_bool = [x in s50.intersection(s0) for x in n50.Symbol]
n50.loc[slice_bool]

,Symbol,Open,High,Low,Last Traded Price,Change,%Change,Traded Volume(lacs),Traded Value(crs),52 Week High,52 Week Low,365 Days % Change,30 Days % Change
2,TATAMOTORS,424.00,437.45,423.50,435.25,11.75,2.77,74.23,320.82,536.70,366.20,-13.55,9.76
13,TECHM,1062.00,1071.95,1055.45,1065.80,7.10,0.67,7.86,83.67,1838.00,943.70,-29.97,4.91
19,INFY,1575.15,1581.80,1568.45,1576.80,6.70,0.43,10.96,172.52,1953.90,1355.00,-9.03,10.37
22,TCS,3324.00,3341.85,3309.00,3323.35,7.40,0.22,4.01,133.31,4043.00,2926.10,-4.73,7.18
27,M&M,1304.00,1309.70,1286.20,1287.35,0.25,0.02,12.76,165.71,1364.85,671.15,39.21,3.16
32,RELIANCE,2630.75,2642.00,2618.85,2626.05,-5.75,-0.22,15.48,407.37,2856.15,2180.00,2.80,10.56
34,HDFCBANK,1612.00,1625.95,1598.00,1606.90,-4.25,-0.26,35.65,574.13,1722.10,1271.60,3.78,13.98
38,WIPRO,402.65,402.65,396.60,398.25,-1.45,-0.36,22.86,91.41,726.80,372.40,-37.95,-2.38
42,SBIN,603.95,603.95,594.10,597.80,-3.50,-0.58,37.39,223.83,622.70,425.00,17.31,11.99
49,ICICIBANK,905.00,906.55,893.00,894.55,-12.00,-1.32,31.64,284.44,943.25,642.15,15.96,3.02


In [94]:
s0 = set(foli0.index)
n50 = n50.set_index('Symbol')
n50 = n50.sort_index()

In [95]:
slice_bool_n50 = [x in s50.intersection(s0) for x in n50.index]
slice_bool_foli0 = [x in s50.intersection(s0) for x in foli0.index]
n50.loc[slice_bool_n50]['Last Traded Price']
foli0['CMP']=n50['Last Traded Price']
foli0

,Price,Qty,Amount,cum_qty,cum_Amt,cum_Avg,Profit,%Profit,cum_profit,Invested_amt,apparent_avg,apparent_amt,CMP
Stock,,,,,,,,,,,,,
BANKBARODA,139.45,5,697.25,5.0,697.25,139.45,0,0.000000,0.00,697.25,139.45,697.25,NaN
HDFCBANK,1401.95,1,1401.95,1.0,1401.95,1401.95,0,0.000000,0.00,1401.95,1401.95,1401.95,1606.90
ICICIBANK,855.15,2,1710.3,7.0,6262.05,894.579,0,0.000000,0.00,6262.05,894.579,6262.05,894.55
INFY,1520.1,-2,-3040.2,0.0,-173.00,0,173,5.690415,173.00,0.00,0,0.00,1576.80
M&M,1334.85,1,1334.85,6.0,7757.60,1292.93,0,0.000000,0.00,7757.60,1292.93,7757.60,1287.35
NATIONALUM,71.4,10,714,30.0,2189.35,72.9783,0,0.000000,0.00,2189.35,72.9783,2189.35,NaN
PNB,36.3,3,108.9,3.0,108.90,36.3,0,0.000000,0.00,108.90,36.3,108.90,NaN
RELIANCE,2434,-5,-12170,0.0,15.40,0,-15.4,-0.126541,-15.40,0.00,0,0.00,2626.05
SBIN,529,2,1058,5.0,2725.80,545.16,0,0.000000,0.00,2725.80,545.16,2725.80,597.80


In [82]:
# foli0 = foli0.fillna(0)
# foli0

,Price,Qty,Amount,cum_qty,cum_Amt,cum_Avg,Profit,%Profit,cum_profit,Invested_amt,apparent_avg,apparent_amt,CMP
Stock,,,,,,,,,,,,,
BANKBARODA,139.45,5.0,697.25,5.0,697.25,139.450000,0.0,0.000000,0.00,697.25,139.450000,697.25,0.00
HDFCBANK,1401.95,1.0,1401.95,1.0,1401.95,1401.950000,0.0,0.000000,0.00,1401.95,1401.950000,1401.95,1606.90
ICICIBANK,855.15,2.0,1710.30,7.0,6262.05,894.578571,0.0,0.000000,0.00,6262.05,894.578571,6262.05,894.55
INFY,1520.10,-2.0,-3040.20,0.0,-173.00,0.000000,173.0,5.690415,173.00,0.00,0.000000,0.00,1576.80
M&M,1334.85,1.0,1334.85,6.0,7757.60,1292.933333,0.0,0.000000,0.00,7757.60,1292.933333,7757.60,1287.35
NATIONALUM,71.40,10.0,714.00,30.0,2189.35,72.978333,0.0,0.000000,0.00,2189.35,72.978333,2189.35,0.00
PNB,36.30,3.0,108.90,3.0,108.90,36.300000,0.0,0.000000,0.00,108.90,36.300000,108.90,0.00
RELIANCE,2434.00,-5.0,-12170.00,0.0,15.40,0.000000,-15.4,-0.126541,-15.40,0.00,0.000000,0.00,2626.05
SBIN,529.00,2.0,1058.00,5.0,2725.80,545.160000,0.0,0.000000,0.00,2725.80,545.160000,2725.80,597.80


In [96]:
foli0['CMP_profit'] = foli0['CMP']*foli0['cum_qty']-foli0['apparent_amt']
foli0['CMP_diff'] = foli0['CMP']-foli0['apparent_avg']
foli0

,Price,Qty,Amount,cum_qty,cum_Amt,cum_Avg,Profit,%Profit,cum_profit,Invested_amt,apparent_avg,apparent_amt,CMP,CMP_profit,CMP_diff
Stock,,,,,,,,,,,,,,,
BANKBARODA,139.45,5,697.25,5.0,697.25,139.45,0,0.000000,0.00,697.25,139.45,697.25,NaN,NaN,NaN
HDFCBANK,1401.95,1,1401.95,1.0,1401.95,1401.95,0,0.000000,0.00,1401.95,1401.95,1401.95,1606.90,204.95,204.95
ICICIBANK,855.15,2,1710.3,7.0,6262.05,894.579,0,0.000000,0.00,6262.05,894.579,6262.05,894.55,-0.20,-0.0285714
INFY,1520.1,-2,-3040.2,0.0,-173.00,0,173,5.690415,173.00,0.00,0,0.00,1576.80,0.00,1576.8
M&M,1334.85,1,1334.85,6.0,7757.60,1292.93,0,0.000000,0.00,7757.60,1292.93,7757.60,1287.35,-33.50,-5.58333
NATIONALUM,71.4,10,714,30.0,2189.35,72.9783,0,0.000000,0.00,2189.35,72.9783,2189.35,NaN,NaN,NaN
PNB,36.3,3,108.9,3.0,108.90,36.3,0,0.000000,0.00,108.90,36.3,108.90,NaN,NaN,NaN
RELIANCE,2434,-5,-12170,0.0,15.40,0,-15.4,-0.126541,-15.40,0.00,0,0.00,2626.05,0.00,2626.05
SBIN,529,2,1058,5.0,2725.80,545.16,0,0.000000,0.00,2725.80,545.16,2725.80,597.80,263.20,52.64


In [84]:
foli0['CMP']

Stock
BANKBARODA       0.00
HDFCBANK      1606.90
ICICIBANK      894.55
INFY          1576.80
M&M           1287.35
NATIONALUM       0.00
PNB              0.00
RELIANCE      2626.05
SBIN           597.80
TATAMOTORS     435.25
TCS           3323.35
TECHM         1065.80
WIPRO          398.25
Total            0.00
Name: CMP, dtype: float64

Stock
BANKBARODA        0.00
HDFCBANK       1606.90
ICICIBANK      6261.85
INFY              0.00
M&M            7724.10
NATIONALUM        0.00
PNB               0.00
RELIANCE          0.00
SBIN           2989.00
TATAMOTORS     2611.50
TCS               0.00
TECHM         10658.00
WIPRO          1991.25
Total             0.00
dtype: float64